In [ ]:
# install required packages
%pip install biopython
# %pip install tensorflow

In [9]:
from Bio import SeqIO

data_file = "sample_datasets/small_sequences.fasta"
processed_output_directory = "full_datasets/preprocessed/small_sequences/"
codon_input_length = 5 # number of codons to input to the model

In [11]:
# map the string to a one-hot encoding by codons
def getMapping():
    chars = ["A", "C", "G", "T"]
    mapping = {}
    for i in range(64):
        seq = chars[(i//16)%4] + chars[(i//4)%4] + chars[i%4]
        arr = [0]*64
        arr[i] = 1
        mapping[seq] = arr
    return mapping

def encode(seq, mapping):
    output = []
    for i in range(0, len(seq), 3):
        codon = seq[i:i+3]
        if codon in mapping:
            output.append(mapping[codon])
        elif len(seq) - i > 3:
            raise Exception("You should not be here")
    return output
    # return [mapping[seq[i:i+3]] for i in range(0, len(seq), 3)]
# select and generate samples for dataset.
def helper():
    mapping = getMapping()
    sequences = SeqIO.parse(data_file, "fasta")
    # for seq in sequences:
    seq = next(sequences)
    strings = str(seq.seq).split("N")
    for st in strings:
        if len(st) < codon_input_length + 1:
            continue
        encoded = encode(st, mapping)
        for i in range(len(encoded) - codon_input_length):
            yield (encoded[i:i+codon_input_length], encoded[i+codon_input_length])
count = 0
for (x, y) in helper():
    x_filename = processed_output_directory + str(count) + '_x.txt'
    y_filename = processed_output_directory + str(count) + '_y.txt'
    with open(x_filename, 'w') as x_file:
        x_file.write(str(x))
    with open(y_filename, 'w') as y_file:
        y_file.write(str(y))
    count += 1